In [5]:
from PIL import Image, ImageDraw, ImageFont
import os

# List of filenames in order
steps = [0,10,20,40,50,100,200,400,800,1600]
filenames = [f"{step}.jpg" for step in steps]

# Load images
images = [Image.open(name) for name in filenames]
width, height = images[0].size

# Font setup (larger font for better visibility)
try:
    font = ImageFont.truetype("arial.ttf", 24)
except IOError:
    font = ImageFont.load_default()

# Additional height for x-axis labels
axis_height = 40
canvas = Image.new("RGB", (width * len(images), height + axis_height), "white")
draw = ImageDraw.Draw(canvas)

# Draw images
for i, (img, step) in enumerate(zip(images, steps)):
    x = i * width
    canvas.paste(img, (x, 0))

    # Draw tick line
    tick_y_start = height + 5
    tick_y_end = height + 15
    tick_x = x + width // 2
    draw.line([(tick_x, tick_y_start), (tick_x, tick_y_end)], fill="black", width=2)

    # Draw label
    label = f"t={step}"
    bbox = draw.textbbox((0, 0), label, font=font)
    text_width = bbox[2] - bbox[0]
    text_x = tick_x - text_width // 2
    draw.text((text_x, tick_y_end + 2), label, fill="black", font=font)
# Save output
canvas.save("timeline.jpg", "JPEG", quality=90)
print("Saved as timeline.png")


Saved as timeline.png


In [10]:
from PIL import Image, ImageDraw, ImageFont
import random

# Load the image and convert to RGBA for transparency
image = Image.open("40.jpg").convert("RGBA")

# Grid size (e.g., 4x4)
rows, cols = 28, 28
block_w = image.width // cols
block_h = image.height // rows

# Overlay for translucent colored blocks
overlay = Image.new("RGBA", image.size, (255, 255, 255, 0))
draw = ImageDraw.Draw(overlay)

# Font setup: try very large size, fallback to default
try:
    font = ImageFont.truetype("arial.ttf", size=block_h // 2)
except IOError:
    font = ImageFont.load_default()

# Generate a list of distinct translucent colors
def random_translucent_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 100)

# Draw each block
for i in range(rows):
    for j in range(cols):
        index = i * cols + j + 1
        x0, y0 = j * block_w, i * block_h
        x1, y1 = x0 + block_w, y0 + block_h

        # Random translucent color
        color = random_translucent_color()
        draw.rectangle([x0, y0, x1, y1], fill=color, outline=(0, 0, 0, 200))

        # Huge centered label
        label = f"d{index}"
        bbox = draw.textbbox((0, 0), label, font=font)
        text_w, text_h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        text_x = x0 + (block_w - text_w) // 2
        text_y = y0 + (block_h - text_h) // 2
        draw.text((text_x, text_y), label, font=font, fill=(0, 0, 0, 255))

# Composite overlay and save
result = Image.alpha_composite(image, overlay).convert("RGB")
result.save("annotated_blocks_small_d.jpg", "JPEG")
print("Saved as annotated_blocks_large_r.jpg")


Saved as annotated_blocks_large_r.jpg


In [13]:
from PIL import Image

# Load your images (ensure they have an alpha channel)
img2 = Image.open("annotated_blocks_small_d.jpg").convert("RGBA")
img1 = Image.open("annotated_blocks_large_r.jpg").convert("RGBA")

# Set gap size
gap = 500  # transparent space between images

# Calculate new image size
new_width = img1.width + gap + img2.width
new_height = max(img1.height, img2.height)

# Create transparent background
combined = Image.new("RGBA", (new_width, new_height), (0, 0, 0, 0))

# Paste images with spacing
combined.paste(img1, (0, 0), mask=img1)
combined.paste(img2, (img1.width + gap, 0), mask=img2)

# Save or show
combined.save("combined.png")
combined.show()
